In [5]:
import pandas as pd
import numpy as np

In [6]:
df1=pd.read_excel('./data/1.xlsx',sheet_name='四个自变量99-17',
keep_default_na=False,
    dtype={
        'COMPANY': str,
        'YEAR': str,
        'ID': str,
    })
df2=pd.read_excel('./data/1.xlsx',sheet_name='违规信息黑名单94-20',keep_default_na=False,
    dtype={
        'COMPANY': str,
        'YEAR': str,
        'YEARS': str,
        'ID': str,
        'CHEAT': int
    })
# df1.set_index('ID')
# df2.set_index('ID')
df1=df1.dropna()
df2=df2.dropna()
df1.head()
# df2.head()

,COMPANY,DATE,YEAR,ID,GAIN,LOSS,TATA1,TATA2,CHCS,OTHREC
0,000001,2014-12-31,2014,20140000001,1.984100e+10,0.0,-9.147210e-07,0.000440,0.087320,0.0
1,000001,2013-12-31,2013,20130000001,1.516600e+10,0.0,2.666105e-04,0.000696,-0.803567,0.0
2,000001,2012-12-31,2012,20120000001,1.338522e+10,0.0,-9.296631e-04,-0.000479,3.187515,0.0
3,000001,2011-12-31,2011,20110000001,1.017927e+10,0.0,3.980625e-04,0.000758,-4.228793,0.0
4,000001,2010-12-31,2010,20100000001,6.143737e+09,0.0,-2.697994e-05,0.000207,1.169672,0.0


In [2]:
print(df1.isnull().sum())
print(df1.shape)
print(df2.info())
# df1.iloc[31520:]
# df1[df1["GAIN"] > 0.5]


NameError: name 'df1' is not defined

In [7]:
cheat_ids=[]
for index, row in df2.iterrows():
    COMPANY=row['COMPANY']
    YEARS=row['YEARS'].strip(" ").split(",")
    for YEAR in YEARS:
        cheat_ids.append(YEAR+COMPANY)
#  show the first 50 items
print(len(cheat_ids))
print(cheat_ids[:50])


1009
['2012000007', '2013000007', '2014000007', '2015000010', '2016000010', '2001000013', '2014000020', '2015000020', '2011000033', '2012000033', '2013000033', '2014000033', '2003000034', '2004000034', '2004000035', '1999000037', '2000000037', '2001000037', '2002000037', '2003000038', '2004000038', '2006000040', '2007000040', '2008000040', '2009000040', '2010000040', '2002000156', '2003000156', '2004000156', '2007000403', '2008000403', '2009000403', '2010000403', '2011000403', '2012000403', '2013000403', '2012000408', '2013000408', '2017000408', '2018000408', '2019000408', '2000000409', '2001000409', '2002000409', '2003000409', '2004000409', '2003000421', '2004000421', '2003000498', '2004000498']


In [8]:
df3=pd.merge(df1,df2[["ID","CHEAT"]],how="left",on="ID")
for index, row in df3.iterrows():
    if index%100==0:
        print(index)
    if row["ID"] in cheat_ids:
        df3["CHEAT"].iloc[index]=1
        cheat_ids.remove(row["ID"])
    else:
        df3["CHEAT"].iloc[index]=0

print(df3.isnull().any())
# df3["CHEAT"]=df3["CHEAT"].fillna(0)
df3.to_csv("./data/1.csv",index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [38]:
df3.isnull().any()


COMPANY    False
DATE       False
YEAR       False
ID         False
GAIN       False
LOSS       False
TATA1      False
TATA2      False
CHCS       False
OTHREC     False
CHEAT       True
dtype: bool